In [65]:
#Import packages used for Analysis and Load the Heart Disease Data file
#Data Source: https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease- cited in my paper, just noting for my records

import pandas as pd
import numpy as np

from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sb
import scipy
from scipy.stats import spearmanr

from pylab import rcParams
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from statsmodels.discrete.discrete_model import Probit

from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from flask import Flask, request

hd = pd.read_csv('heart_2020_cleaned.csv')

In [66]:
#Convert variables to useable form for analysis- Yes(1)/No(0)
yes_no_dict={"Yes":1,"No":0}
hd['HeartDiseaseB'] = hd['HeartDisease'].map(yes_no_dict)
hd['SmokingB'] = hd['Smoking'].map(yes_no_dict)
hd['StrokeB'] = hd['Stroke'].map(yes_no_dict)
hd['DiffWalkingB'] = hd['DiffWalking'].map(yes_no_dict)
hd['PhysicalActivityB'] = hd['PhysicalActivity'].map(yes_no_dict)
hd['AsthmaB'] = hd['Asthma'].map(yes_no_dict)
hd['AlcoholDrinkingB'] = hd['AlcoholDrinking'].map(yes_no_dict)
hd['KidneyDiseaseB'] = hd['KidneyDisease'].map(yes_no_dict)
hd['SkinCancerB'] = hd['SkinCancer'].map(yes_no_dict)

#Diabetic [No, Yes, 'No, borderline diabetes', 'Yes (during pregnancy)']
diab_dict={"Yes":1,"No":0,"No, borderline diabetes":0,"Yes (during pregnancy)":0}
hd['DiabB'] = hd['Diabetic'].map(diab_dict)
diab_preg_dict={"Yes":0,"No":0,"No, borderline diabetes":0,"Yes (during pregnancy)":1}
hd['DiabPregB'] = hd['Diabetic'].map(diab_preg_dict)

#Male/Female Dict
mf_dict={"Female":1,"Male":0}
hd['FemaleB'] = hd['Sex'].map(mf_dict)

#RaceDict ['White' 'Black' 'Asian' 'American Indian/Alaskan Native' 'Other''Hispanic']
othrnonwhite_dict = {"White":0, "Black":0,"Asian":0,"American Indian/Alaskan Native":0,"Hispanic":0,"Other":1}
hd['Othr_non-whiteB'] = hd['Race'].map(othrnonwhite_dict)

black_dict = {"White":0, "Black":1,"Asian":0,"American Indian/Alaskan Native":0,"Hispanic":0,"Other":0}
hd['BlackB'] = hd['Race'].map(black_dict)

asian_dict = {"White":0, "Black":0,"Asian":1,"American Indian/Alaskan Native":0,"Hispanic":0,"Other":0}
hd['AsianB'] = hd['Race'].map(asian_dict)

AI_dict = {"White":0, "Black":0,"Asian":0,"American Indian/Alaskan Native":1,"Hispanic":0,"Other":0}
hd['AI_ANB'] = hd['Race'].map(AI_dict)

hispanic_dict = {"White":0, "Black":0,"Asian":0,"American Indian/Alaskan Native":0,"Hispanic":1,"Other":0}
hd['HispanicB'] = hd['Race'].map(hispanic_dict)

white_dict = {"White":1, "Black":0,"Asian":0,"American Indian/Alaskan Native":0,"Hispanic":0,"Other":0}
hd['WhiteB'] = hd['Race'].map(white_dict)

#Age
O55_dict = {'55-59':1, '80 or older':1, '65-69':1, '75-79':1, '40-44':0, '70-74':1, '60-64':1, '50-54':0,
 '45-49':0, '18-24':0, '35-39':0, '30-34':0, '25-29':0}
hd['Over55B'] = hd['AgeCategory'].map(O55_dict)

#GenHealth['Poor'=1,'Fair'=2,'Very good'=3,'Good'=4, 'Excellent'=5]
genhlth_dict = {'Poor':1,'Fair':2,'Very good':3,'Good':4, 'Excellent':5}
hd['GenHealthCtgry'] = hd['GenHealth'].map(genhlth_dict)

print(hd.columns)

Index(['HeartDisease', 'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
       'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
       'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime',
       'Asthma', 'KidneyDisease', 'SkinCancer', 'HeartDiseaseB', 'SmokingB',
       'StrokeB', 'DiffWalkingB', 'PhysicalActivityB', 'AsthmaB',
       'AlcoholDrinkingB', 'KidneyDiseaseB', 'SkinCancerB', 'DiabB',
       'DiabPregB', 'FemaleB', 'Othr_non-whiteB', 'BlackB', 'AsianB', 'AI_ANB',
       'HispanicB', 'WhiteB', 'Over55B', 'GenHealthCtgry'],
      dtype='object')


In [67]:
#removing duplicate columns (non-binary)
#Removing White column created from Race to remove collinearity (White=Reference Category)
hdb = hd.drop(columns=['PhysicalActivity','Diabetic','GenHealth','HeartDisease',
                       'Smoking', 'AlcoholDrinking', 'Stroke','DiffWalking', 'Sex', 
                       'AgeCategory','Race', 'Diabetic','Asthma', 'KidneyDisease', 'SkinCancer','WhiteB'], axis=1)

In [68]:
hdb.describe(include='all').T.style.background_gradient(cmap='Purples')

In [69]:
#Rearranging columns to make HeartDiseaseBinary earier to review on correlation table
hdb = hdb[['HeartDiseaseB','BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime',
       'SmokingB', 'StrokeB', 'DiffWalkingB', 'PhysicalActivityB', 'AsthmaB',
       'AlcoholDrinkingB', 'KidneyDiseaseB', 'SkinCancerB', 'DiabB',
       'DiabPregB', 'FemaleB', 'Othr_non-whiteB', 'BlackB', 'AsianB', 'AI_ANB',
       'HispanicB', 'Over55B', 'GenHealthCtgry']]

In [70]:
hdb['log_BMI'] = np.log10(hdb['BMI'])
print(hdb.head(n=10))

   HeartDiseaseB    BMI  PhysicalHealth  MentalHealth  SleepTime  SmokingB  \
0              0  16.60             3.0          30.0        5.0         1   
1              0  20.34             0.0           0.0        7.0         0   
2              0  26.58            20.0          30.0        8.0         1   
3              0  24.21             0.0           0.0        6.0         0   
4              0  23.71            28.0           0.0        8.0         0   
5              1  28.87             6.0           0.0       12.0         1   
6              0  21.63            15.0           0.0        4.0         0   
7              0  31.64             5.0           0.0        9.0         1   
8              0  26.45             0.0           0.0        5.0         0   
9              0  40.69             0.0           0.0       10.0         0   

   StrokeB  DiffWalkingB  PhysicalActivityB  AsthmaB  ...  DiabPregB  FemaleB  \
0        0             0                  1        1  ...   

In [71]:
hdb['log_SleepTime'] = np.log10(hdb['SleepTime'])
print(hdb.head(n=10))

   HeartDiseaseB    BMI  PhysicalHealth  MentalHealth  SleepTime  SmokingB  \
0              0  16.60             3.0          30.0        5.0         1   
1              0  20.34             0.0           0.0        7.0         0   
2              0  26.58            20.0          30.0        8.0         1   
3              0  24.21             0.0           0.0        6.0         0   
4              0  23.71            28.0           0.0        8.0         0   
5              1  28.87             6.0           0.0       12.0         1   
6              0  21.63            15.0           0.0        4.0         0   
7              0  31.64             5.0           0.0        9.0         1   
8              0  26.45             0.0           0.0        5.0         0   
9              0  40.69             0.0           0.0       10.0         0   

   StrokeB  DiffWalkingB  PhysicalActivityB  AsthmaB  ...  FemaleB  \
0        0             0                  1        1  ...        1   
1

In [72]:
#1Indeps for Regression1(hdb1, x)
hdb1= hdb.drop(['log_BMI','log_SleepTime'],axis=1)
x= hdb1.drop(['HeartDiseaseB'],axis=1)
x.head(n=2)

BMI  PhysicalHealth  MentalHealth  SleepTime  SmokingB  StrokeB  \
0  16.60             3.0          30.0        5.0         1        0   
1  20.34             0.0           0.0        7.0         0        1   

   DiffWalkingB  PhysicalActivityB  AsthmaB  AlcoholDrinkingB  ...  DiabB  \
0             0                  1        1                 0  ...      1   
1             0                  1        0                 0  ...      0   

   DiabPregB  FemaleB  Othr_non-whiteB  BlackB  AsianB  AI_ANB  HispanicB  \
0          0        1                0       0       0       0          0   
1          0        1                0       0       0       0          0   

   Over55B  GenHealthCtgry  
0        1               3  
1        1               3  

[2 rows x 22 columns]

In [73]:
#y can be used for all regressions
y=hdb["HeartDiseaseB"]
print(y)

0         0
1         0
2         0
3         0
4         0
         ..
319790    1
319791    0
319792    0
319793    0
319794    0
Name: HeartDiseaseB, Length: 319795, dtype: int64


In [74]:
#1A Probit 1-- Create Test/Train Variables using test_train_split from sklearn 25% in test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


print('Shape of training x:', X_train.shape)
print('Shape of testing x:', X_test.shape)
print('Shape of training y:', y_train.shape)
print('Shape of training y:', y_test.shape)

Shape of training x: (255836, 22)
Shape of testing x: (63959, 22)
Shape of training y: (255836,)
Shape of training y: (63959,)


In [75]:
#1B logistic regression1
#max_iter=100000000
y_train= y_train.values.reshape(-1,1)

model1 = Probit(y_train,x_train)
probit_model1 = model1.fit()
print(probit_model1.summary())

#y_predictions = probit_reg.predict(x_test)
#y_predictions[0:10]

Optimization terminated successfully.
         Current function value: 0.240523
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:               255836
Model:                         Probit   Df Residuals:                   255814
Method:                           MLE   Df Model:                           21
Date:                Sat, 07 May 2022   Pseudo R-squ.:                  0.1766
Time:                        14:34:52   Log-Likelihood:                -61534.
converged:                       True   LL-Null:                       -74733.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
BMI                  -0.0166      0.001    -31.182      0.000      -0.018      -0.016
Physical

In [76]:
y1_predictions = probit_model1.predict(x_test)
y1_predictions[0:10]

278727    0.143983
219612    0.420804
99736     0.004444
127679    0.043239
174964    0.132332
101103    0.002742
127527    0.376327
160253    0.032317
60088     0.124775
303932    0.059212
dtype: float64

In [77]:
hd_Probit = pd.DataFrame(x_test)
hd_Probit["HD Actuals"]= y_test
hd_Probit["HD Predictions"] = y1_predictions

hd_Probit.head(n=15)
hd_Probit.to_csv(r"Probit HD model1 results.csv")

In [78]:
#---End of Regression 1----

In [79]:
#2Indeps for Regression2(hdb2, l)
hdb2= hdb.drop(['BMI','SleepTime'],axis=1)
l= hdb2.drop(['HeartDiseaseB'],axis=1)
l.head(n=2)


PhysicalHealth  MentalHealth  SmokingB  StrokeB  DiffWalkingB  \
0             3.0          30.0         1        0             0   
1             0.0           0.0         0        1             0   

   PhysicalActivityB  AsthmaB  AlcoholDrinkingB  KidneyDiseaseB  SkinCancerB  \
0                  1        1                 0               0            1   
1                  1        0                 0               0            0   

   ...  FemaleB  Othr_non-whiteB  BlackB  AsianB  AI_ANB  HispanicB  Over55B  \
0  ...        1                0       0       0       0          0        1   
1  ...        1                0       0       0       0          0        1   

   GenHealthCtgry   log_BMI  log_SleepTime  
0               3  1.220108       0.698970  
1               3  1.308351       0.845098  

[2 rows x 22 columns]

In [80]:
#y can be used for all regressions
y2=hdb["HeartDiseaseB"]
print(y2)

0         0
1         0
2         0
3         0
4         0
         ..
319790    1
319791    0
319792    0
319793    0
319794    0
Name: HeartDiseaseB, Length: 319795, dtype: int64


In [81]:
#2A Probit 1-- Create Test/Train Variables using test_train_split from sklearn 25% in test
l_train, l_test, y2_train, y2_test = train_test_split(l, y2, test_size=0.2)


print('Shape of training x:', l_train.shape)
print('Shape of testing x:', l_test.shape)
print('Shape of training y:', y2_train.shape)
print('Shape of training y:', y2_test.shape)

Shape of training x: (255836, 22)
Shape of testing x: (63959, 22)
Shape of training y: (255836,)
Shape of training y: (63959,)


In [82]:
#1B logistic regression1
#max_iter=100000000
y2_train= y2_train.values.reshape(-1,1)

model2 = Probit(y2_train,l_train)
probit_model2 = model2.fit()
print(probit_model2.summary())

#y_predictions = probit_reg.predict(x_test)
#y_predictions[0:10]

Optimization terminated successfully.
         Current function value: 0.237366
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:               255836
Model:                         Probit   Df Residuals:                   255814
Method:                           MLE   Df Model:                           21
Date:                Sat, 07 May 2022   Pseudo R-squ.:                  0.1881
Time:                        14:34:58   Log-Likelihood:                -60727.
converged:                       True   LL-Null:                       -74799.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
PhysicalHealth        0.0046      0.000      9.243      0.000       0.004       0.006
MentalHe

In [83]:
y2_predictions = probit_model2.predict(l_test)
y2_predictions[0:10]

248751    0.463476
145475    0.004870
279619    0.037467
261185    0.013166
214682    0.131287
59538     0.024273
10252     0.009927
296643    0.065974
156617    0.018271
204016    0.033834
dtype: float64

In [84]:
hd_Probit2 = pd.DataFrame(l_test)
hd_Probit2["HD Actuals"]= y2_test
hd_Probit2["HD Predictions"] = y2_predictions

hd_Probit2.head(n=15)
hd_Probit2.to_csv(r"Probit HD model2 results.csv")

In [ ]:
#---End of Regression 2----

In [75]:
#1Indeps for Regression3(hdb1, x)
hdb3= hdb.drop(['log_BMI','log_SleepTime','DiffWalkingB','DiabPregB'],axis=1)
m= hdb3.drop(['HeartDiseaseB'],axis=1)
m.head(n=2)

BMI  PhysicalHealth  MentalHealth  SleepTime  SmokingB  StrokeB  \
0  16.60             3.0          30.0        5.0         1        0   
1  20.34             0.0           0.0        7.0         0        1   

   PhysicalActivityB  AsthmaB  AlcoholDrinkingB  KidneyDiseaseB  SkinCancerB  \
0                  1        1                 0               0            1   
1                  1        0                 0               0            0   

   DiabB  FemaleB  Othr_non-whiteB  BlackB  AsianB  AI_ANB  HispanicB  \
0      1        1                0       0       0       0          0   
1      0        1                0       0       0       0          0   

   Over55B  GenHealthCtgry  
0        1               3  
1        1               3

In [76]:
#y can be used for all regressions
y3=hdb["HeartDiseaseB"]
print(y3)

0         0
1         0
2         0
3         0
4         0
         ..
319790    1
319791    0
319792    0
319793    0
319794    0
Name: HeartDiseaseB, Length: 319795, dtype: int64


In [77]:
#2A Probit 1-- Create Test/Train Variables using test_train_split from sklearn 25% in test
m_train, m_test, y3_train, y3_test = train_test_split(m, y3, test_size=0.2)
print(m_train.shape,y3_train.shape)
print(m_test.shape,y3_train.shape)

"m_train"
print(m_train)
"y2_train"
print(y3_train)

m_train, m_test, y3_train, y3_test = train_test_split(m, y3, shuffle = True, test_size = .2)

print('Shape of training x:', m_train.shape)
print('Shape of testing x:', m_test.shape)
print('Shape of training y:', y3_train.shape)
print('Shape of training y:', y3_test.shape)

(255836, 20) (255836,)
(63959, 20) (255836,)
          BMI  PhysicalHealth  MentalHealth  SleepTime  SmokingB  StrokeB  \
139000  26.50             0.0           0.0        7.0         1        0   
193289  25.80             0.0           0.0        6.0         0        0   
102271  35.29             2.0           3.0        6.0         0        0   
284107  30.68             0.0           0.0        8.0         1        0   
138186  23.18             2.0           0.0        8.0         1        0   
...       ...             ...           ...        ...       ...      ...   
144420  21.46             0.0           0.0        6.0         1        0   
73193   23.62            12.0           5.0        5.0         1        0   
62442   21.22             0.0           0.0        7.0         0        0   
130809  26.31             0.0           0.0        7.0         0        0   
232906  36.61             5.0           5.0        7.0         1        0   

        PhysicalActivityB  Ast

In [78]:
#1B logistic regression1
#max_iter=100000000
y3_train= y3_train.values.reshape(-1,1)

model3 = Probit(y3_train,m_train)
probit_model3 = model.fit()
print(probit_model3.summary())



Optimization terminated successfully.
         Current function value: 0.241927
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:               255836
Model:                         Probit   Df Residuals:                   255816
Method:                           MLE   Df Model:                           19
Date:                Fri, 06 May 2022   Pseudo R-squ.:                  0.1745
Time:                        22:55:26   Log-Likelihood:                -61894.
converged:                       True   LL-Null:                       -74977.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
BMI                  -0.0146      0.001    -27.995      0.000      -0.016      -0.014
Physical

In [79]:
y3_predictions = probit_model3.predict(m_test)
y3_predictions[0:10]

167925    0.100101
213862    0.018283
36378     0.009353
201591    0.031138
131939    0.012465
105781    0.077351
124479    0.020032
118767    0.094808
215502    0.029904
51446     0.027916
dtype: float64